# Meta-predicates:

Meta-predicates are built in predicates which evaluation directly interfere with the Prolog interpreter underneath.

- # call(T). 
  Explanation: In Prolog predicates (i.e., programs!) and terms (i.e.,data!) have the same syntactical structure. As a consequence, predicates and terms can be exchanged and exploited in different roles! Because of the non-distincion between terms and predicate syntax we can pass a predicate as input to another predicate (also if in logic doesn't make sense). If you think about it i's like if the predicate/program/function is treated as a data/parameter, SO THIS IS LIKE PASSING TO A HIGH ORDER FUNTION, A FUNCTION AS PARAMETER. But once you have the predicate passed as parameter, how can you use it? WITH THE META-PREDICATE "call": when writing call(T) the Prolog interpreter is requested to evaluate (execute) T (T must be a predicate, or anyway not a constant).

  Examples: 

1) 
  p(X):- call(X). 
  
  q(a).
  
  query :- p(q(Y)). 
  
  answer: yes Y = a.
  
2) Define a program that behaves as a procedural if_then_else construct if_then_else(Cond,Goal1,Goal2) – If Cond is evaluated to true, then Goal1 is evaluated – If Cond is evaluated to false, then Goal2, is evaluated

if_then_else(Cond,Goal1,Goal2):

call(Cond), !, call(Goal1). 

if_then_else(Cond,Goal1,Goal2):

call(Goal2).

The main point about call is that it permits you to pass programs as terms, and execute them!!!!!!!!!!

(A side effect is that you can give exactly the same name for a predicate and for a term in the same KB).


- # Fail

It can be used to impose a backtracking, indeed its evaluation is always false. Why would we need it? Different reasons:

- to create iterations! thanks to fail you backtrack to the last choice point, so this can be useful to iterate on some terms of your knowledge base.

Example1: apply q(x) on all the x which satisfy p(X):

p(1).

p(2).

p(3).

q(X) :- integer(X), write(X), write('is an integer'), nl.  %nl means new line.

q(X) :- write(X), write('is NOT an integer'), nl.
 
iterate :- call(p(X)), verify(q(X)), fail.

iterate.

verify(q(X)) :- call(q(X)),!.


query: iterate.

IN THIS EXAMPLE THERE ARE MANY IMPORTANT THINGS:

1) How could you make this code more efficient? 
- You could write q(X) :- integer(X),!,... in the first q preicate, 
- Or you could write q(X) :- \+ integer(X),.. in the second q predicate. (note that \+ is the "negation as failure" in prolog).
  
  Why would this make it more efficient? look point 3).
 
2) Is the result the same if you replace "iterate :- call(p(X)), verify(q(X)), fail." with "iterate :- call(p(X)), call(q(X)),!, fail." ? NO!! because in the second case the cut is going to effect also the predicates iterate and p(X), so when fail is met, the iteration finishes!!

3) Why is useful the cut into the predicate verify?
   Because remember that when you use fail it backtracks to the last choice point, and if you don't use the cut the lsst choice point is q!!! so it would print both "x is an integer" and "X is not an integer" (unless you take the safety measures discussed in the point 1).

Example2: implement the negation as failure:

not(P) :- call(P),!,fail.  % if P is true then the cut and fail are evaluated, so the answer to not(P) is false.

not(P).

Doubt: can you replace call(P) with P? No! because P is passed to the not predicate as a term, so within the predicate not it is considered as a term, unless you use call.

Example3: the combination "!,fail". As you have seen in example2 if you use "!,fail" it happens that the output is false for sure, because none of the other options are going to be evaluated. So we can make efficient programs which are typical of dealing with some exceptions, for instance if you want define the "fly" property which is true for all the birds BUT penguins and chicken:

bird(pigeon).
bird(crow).
bird(parrot).
bird(owl).
bird(seagull).
bird(dove).
bird(penguin).
bird(chicken).

fly(X) :- penguin(X),!,fail.
fly(X) :- chicken(X),!,fail.
fly(X) :- bird(X). 

This is efficient because you can impose the exceptions, instead of verify if X is not something which makes bird true, which could be a long list. Note that penguin and chicken are both birds, but they do not fly, so this is the best way to deal with exceptions.

- # setof and bagof:
  - setof(X,P,S). S is the set of instances X that satisfy the goal P
  - bagof(X,P,L). L is the list of instances X that satisfy the goal P
  
Example:

p(1).

p(2).

p(1).

r(2).

query: setof(X,p(X),S)  answer: S=[1,2], X=X.   Note that X is NOT UNIFIED with any value!!

query: bagof(X,p(X),L) answer: L=[1,2,1], X=X   Note that X is NOT UNIFIED with any value!!

but also different queries (I show it only for setof, but is the same reasoning with bagof):

query: setof(X,p(X),[1,2])  answer:yes, X=X.   Note that X is NOT UNIFIED with any value!!

%%%%%%

query: setof(X, (p(X),r(X)) , S)  answer: S=[2]., X=X.   Note that X is NOT UNIFIED with any value!!

%%%%%

:- setof(p(X), p(X),S). answer: S=[p(1),p(2),p(0),p(1)] X=X


father(giovanni,mario). 
father(giovanni,giuseppe).
father(mario, paola).
father(mario,aldo). 
father(giuseppe,maria).

query: setof((X,Y), father(X,Y), S). 

answer: S=[(giovanni,mario),(giovanni,giuseppe), (mario, paola), (mario,aldo), (giuseppe,maria)] X=X Y=Y

Complex example: use of setof to implement the implication: suppose you want to see if for each father(p,Y) then employee(Y), so if all the sons of p are employee.

father(p,Jack).

father(p,U).

employee(Jack).

employee(U).

imply(Y):- setof(X, father(Y,X), L), verify(L).

verify([]).

verify([H|T]):- employee(H), verify(T).


Iterations with setof:
Iterate a procedure q on all the X which satisfy p.

iterate :- setof(X,p(X),S), apply(S).

apply([])
apply([H|T)] :- q(H), apply(T).

Is it better to implement iteration in this way or using cut and fail? it is better this way because it preserves backtrackability since the cut is not used.



The limits of setof and bagof:

father(giovanni,mario). 
father(giovanni,giuseppe).
father(mario, paola). 
father(mario,aldo). 
father(giuseppe,maria).


query: setof(X, father(X,Y), S).

expected answer: S=[giovanni,mario,giuseppe], X=X.

real answer: X=X Y= aldo S=[mario]. (then if youw ask for the next answer: X=X Y=giuseppe S=[giovanni]....)

I was expecting all the X for which father(X,Y) is true, INSTEAD, it returned those X for which, for the same value of Y, father(X,Y) is true.  (don't worry about the order, it returns the output on the lexicographic order based on Y).

solution? We need to specify that Y has to be quantified existentially in this way:

query: setof(X, Y^father(X,Y), S). 

answer: X=X, Y=Y, S=[mario, giovanni, giuseppe].

- # findall(X,P,S)
It is like setof, in which you don't need to specify that the other variable(s) have to be existentially quantified!
The only difference is if no X satisfies P, then it returns an empty list, instead setof and begof returns "no" or an error.

- # working on terms:

• var(Term) true if Term is currently a free variable

• nonvar(Term) true if Term currently is not a free variable

• number(Term) true if Term is a number

• ground(Term) true if Term holds no free variables

- # =..
The operator "=.." allows you to write any structured term/predicate in the form of a list. The left side or the right side of the operator can be a variable, but not both:

query: foo(Hello,X) =.. L.

answer: L=[foo,Hello,X]

query: T =.. [boo, foo(1)]

answer: T=[boo(foo(1))]

Note this cool thing: you can take a list and convert it into a term and then treat it as a procedure by using call:

Term =.. [father,mario,aldo], call(Term).

- # clause

Note the internal representation of prolog:

h. 

h :- b1, b2, …, bn. 

Correspond to the terms: 

(h, true)

(h, ','(b1, ','(b2, ','( ...','( bn-1, bn) ...)))

That's why it makes sense to use the meta predicate "clause" which is able to ispect the program like this:

p(1).

q(X,a) :- p(X), r(a).

q(2,Y) :- d(Y).

query: - clause(p(1),BODY).   answer: yes BODY=true 

query: - clause(p(X),true). answer: yes X=1 

query: - clause(q(X,Y), BODY).  answer: yes Y=a BODY=p(X),r(a); X=2  BODY=d(Y); no

query: - clause(HEAD,true). answer: Error - invalid key to data-base

More complex:

p(1).

q(X,a) :- p(X), r(a).

q(2,Y) :- d(Y).

test(Body) :- clause(q(_X,_Y),Body).  %_X,_Y are simply mutable variables with a name.

query: test(Body).

answer: Body = ((p(_6020),r(a));  Body = d(_6022).

Why is all this important? because this allows you to know your program during runtime! But not only! You can also modify it runtime: you use clause to take the bodies as data, then you can modify this data, then you can use call.